Bottle-tutorial
===

from

https://dev.opencascade.org/doc/overview/html/occt__tutorial.html

lookup the occ - tutorial for explanations.

In [1]:
from netgen.occ import *

In [2]:
myHeight = 70
myWidth = 50
myThickness = 30

In [3]:
pnt1 = gp_Pnt(-myWidth / 2., 0, 0);
pnt2 = gp_Pnt(-myWidth / 2., -myThickness / 4., 0);
pnt3 = gp_Pnt(0, -myThickness / 2., 0);
pnt4 = gp_Pnt(myWidth / 2., -myThickness / 4., 0);
pnt5 = gp_Pnt(myWidth / 2., 0, 0);

In [4]:
seg1 = Segment(pnt1, pnt2)
arc = ArcOfCircle(pnt2, pnt3, pnt4)
seg2 = Segment(pnt4, pnt5)

In [5]:
print (type(seg1), seg1.type, arc.type, seg2.type)
wire = Wire ([seg1, arc, seg2])

<class 'netgen.libngpy._NgOCC.TopoDS_Edge'> TopAbs_ShapeEnum.EDGE TopAbs_ShapeEnum.EDGE TopAbs_ShapeEnum.EDGE


In [6]:
# trafo = gp_Trsf().SetMirror(gp_Ax1((0,0,0), (1,0,0)))
trafo = gp_Trsf.Mirror(gp_Ax1((0,0,0), (1,0,0)))
mirrored_wire = trafo(wire)

In [7]:
w = Wire([wire, mirrored_wire])
f = Face (w)
body = Prism(f, gp_Vec((0,0,myHeight)))

In [8]:
edges = body.edges
body = body.MakeFillet (edges, myThickness / 12.)

In [9]:
from ngsolve import Mesh
from ngsolve.webgui import Draw

In [10]:
geo = OCCGeometry(body)
mesh = Mesh(geo.GenerateMesh(maxh=5))

In [12]:
Draw (mesh);

WebGuiWidget(value={'ngsolve_version': '6.2.2104-70-gf9a5bdca4', 'mesh_dim': 3, 'order2d': 1, 'order3d': 1, 'd…

Adding the neck:
---

In [13]:
# neckax2 = gp_Ax2( (0,0,myHeight), (0,0,1))
neckax2 = gp_Ax2( body.faces.Max(Z).center, Z)

In [14]:
myNeckRadius = myThickness / 4.
myNeckHeight = myHeight / 10
neck = Cylinder(neckax2, myNeckRadius, myNeckHeight);

In [15]:
body = body+neck

In [17]:
geo = OCCGeometry(body)
mesh = Mesh(geo.GenerateMesh(maxh=5))
Draw (mesh);

WebGuiWidget(value={'ngsolve_version': '6.2.2104-70-gf9a5bdca4', 'mesh_dim': 3, 'order2d': 1, 'order3d': 1, 'd…

find face with maximal z-coordinate:

In [18]:
fmax = body.faces.Max(Z)

In [19]:
thickbody = body.MakeThickSolid([fmax], -myThickness / 50, 1.e-3)

In [20]:
geo = OCCGeometry(thickbody)
mesh = Mesh(geo.GenerateMesh(maxh=3))
Draw (mesh);

WebGuiWidget(value={'ngsolve_version': '6.2.2104-70-gf9a5bdca4', 'mesh_dim': 3, 'order2d': 1, 'order3d': 1, 'd…

In [21]:
cyl1 = Cylinder(neckax2, myNeckRadius * 0.99, 1).faces[0]
cyl2 = Cylinder(neckax2, myNeckRadius * 1.05, 1).faces[0]

In [22]:
import math
aPnt = gp_Pnt2d(2. * math.pi, myNeckHeight / 2.)
aDir = gp_Dir2d( (2. * math.pi, myNeckHeight / 4.) )
anAx2d = gp_Ax2d(aPnt, aDir)

In [23]:
aMajor = 2. * math.pi
aMinor = myNeckHeight / 10
arc1 = Ellipse(anAx2d, aMajor, aMinor).Trim(0, math.pi)
arc2 = Ellipse(anAx2d, aMajor, aMinor/4).Trim(0, math.pi)

In [24]:
seg = Segment(arc1.start, arc1.end)

In [25]:
wire1 = Wire( [Edge(arc1, cyl1), Edge(seg, cyl1)] )
wire2 = Wire( [Edge(arc2, cyl2), Edge(seg, cyl2)] )

In [26]:
threading = ThruSections ([wire1, wire2])

In [27]:
res = thickbody+threading
geo = OCCGeometry(res)
mesh = Mesh(geo.GenerateMesh(maxh=5))

In [28]:
Draw (mesh);

WebGuiWidget(value={'ngsolve_version': '6.2.2104-70-gf9a5bdca4', 'mesh_dim': 3, 'order2d': 1, 'order3d': 1, 'd…